In [6]:
import streamlit as st
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import re
from io import BytesIO
from PIL import Image

# Charger et redimensionner le logo
logo = Image.open("360_capital_vc_logo.jpeg")
logo = logo.resize((64, 64))


# Configuration de la page
st.set_page_config(
    page_title="Nettoyage Données Crunchbase",
    page_icon=logo,
    layout="wide"
)

def get_domain(url):
    """Extrait le domaine d'une URL et le formate"""
    if pd.isna(url):
        return None
    try:
        domain = urlparse(url).netloc
        domain = re.sub(r'^www\d*\.', '', domain).split(':')[0]
        return domain.lower()
    except:
        return None

def clean_crunchbase_data(df):
    """
    Nettoie les données de levées de fonds Crunchbase
    
    Args:
        df: DataFrame avec les colonnes Crunchbase
        
    Returns:
        DataFrame nettoyé avec les colonnes formatées
    """
    # Créer une copie pour ne pas modifier l'original
    df_clean = df.copy()
    
    # 1. Filtrer les types de financement non désirés
    funding_types_to_remove = [
        'Corporate Round',
        'Grant',
        'Post-IPO Debt',
        'Equity Crowdfunding',
        'Debt Financing',
        'Convertible Note',
        'Series C'
    ]
    
    initial_count = len(df_clean)
    df_clean = df_clean[~df_clean['Funding Type'].isin(funding_types_to_remove)]
    filtered_count = initial_count - len(df_clean)
    
    # 2. Convertir les montants USD en devise originale
    mask_usd = df_clean['Money Raised Currency'] == 'USD'
    mask_has_both = pd.notna(df_clean['Money Raised']) & pd.notna(df_clean['Money Raised (in USD)'])
    
    # Calculer le taux de change moyen pour les lignes non-USD
    rates = df_clean[~mask_usd & mask_has_both].apply(
        lambda row: row['Money Raised (in USD)'] / row['Money Raised'] 
        if row['Money Raised'] != 0 else np.nan,
        axis=1
    )
    avg_rate = rates.median() if len(rates) > 0 else 1.0
    
    # Appliquer la conversion inverse pour les montants USD
    df_clean.loc[mask_usd & pd.isna(df_clean['Money Raised']) & pd.notna(df_clean['Money Raised (in USD)']), 'Money Raised'] = \
        df_clean.loc[mask_usd & pd.isna(df_clean['Money Raised']) & pd.notna(df_clean['Money Raised (in USD)']), 'Money Raised (in USD)'] / avg_rate
    
    # 3. Appliquer le formatage des URLs avec get_domain
    df_clean['Website_formatted'] = df_clean['Organization Website'].apply(get_domain)
    
    # 3bis Changer le format des montants 

    df_clean['Money Raised'] = df_clean['Money Raised'].apply(lambda x: f"€M {x:,.0f}" if pd.notna(x) else x)  

    # 4. Créer le nouveau DataFrame avec les colonnes demandées
    df_final = pd.DataFrame({
        'Company Name': df_clean['Organization Name'],
        'Website 2': '',
        'Website': df_clean['Website_formatted'],
        'Description': df_clean['Organization Description'],
        'Secteur': df_clean['Organization Industries'],
        'Date annonce levée': '',
        'Montant': df_clean['Money Raised'],
        'Investisseurs': df_clean['Investor Names']
    })
    
    # Réinitialiser l'index
    df_final = df_final.reset_index(drop=True)
    
    return df_final, filtered_count


# Interface principale
st.title("Nettoyage de Données Crunchbase")
st.markdown("---")

st.markdown("""
### Instructions
        1. Téléchargez votre fichier CSV exporté depuis Crunchbase.
        2. Cliquez sur "Nettoyer les données" pour lancer le processus de nettoyage.
        3. Téléchargez les données nettoyées au format CSV ou Excel.
""")

st.markdown("---")

# Upload du fichier
uploaded_file = st.file_uploader(
    "Chargez votre fichier CSV Crunchbase",
    type=['csv'],
    help="Le fichier doit contenir les colonnes standard de Crunchbase"
)

if uploaded_file is not None:
    try:
        # Lecture du fichier
        df = pd.read_csv(uploaded_file)
        
        st.success(f"✅ Fichier chargé : {len(df)} lignes détectées")
        
        # Afficher un aperçu des données originales
        with st.expander("Aperçu des données originales"):
            st.dataframe(df.head(10), use_container_width=True)
        
        # Bouton de nettoyage
        if st.button("Nettoyer les données", type="primary", use_container_width=True):
            with st.spinner("Nettoyage en cours..."):
                # Nettoyage
                df_clean, filtered_count = clean_crunchbase_data(df)
                
                # Stocker dans session state
                st.session_state['df_clean'] = df_clean
                st.session_state['filtered_count'] = filtered_count
        
        # Afficher les résultats si disponibles
        if 'df_clean' in st.session_state:
            df_clean = st.session_state['df_clean']
            filtered_count = st.session_state['filtered_count']
            
            st.markdown("---")
            st.success("Nettoyage terminé !")
            
            # Statistiques
            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("Lignes initiales", len(df))
            with col2:
                st.metric("Lignes filtrées", filtered_count)
            with col3:
                st.metric("Lignes finales", len(df_clean))
            
            # Aperçu des données nettoyées
            st.subheader("Données nettoyées")
            st.dataframe(df_clean, use_container_width=True)
            
            # Boutons de téléchargement
            st.markdown("---")
            st.subheader("Télécharger les résultats")
            
            col1, col2 = st.columns(2)
            
            with col1:
                # CSV
                csv = df_clean.to_csv(index=False).encode('utf-8')
                st.download_button(
                    label="Télécharger en CSV",
                    data=csv,
                    file_name="crunchbase_cleaned.csv",
                    mime="text/csv",
                    use_container_width=True
                )
            
    
    except Exception as e:
        st.error(f"❌ Erreur lors du traitement du fichier : {str(e)}")
        st.info("Vérifiez que votre fichier contient bien toutes les colonnes requises.")

else:
    st.info("Charger un fichier CSV")

# Footer
st.markdown("---")
st.markdown(
    """
    <div style='text-align: center; color: gray;'>
    Outil de nettoyage de données Crunchbase 360 Capital 
    </div>
    """,
    unsafe_allow_html=True
)

2025-10-13 17:27:43.410 
  command:

    streamlit run /Users/justinkim/Documents/GitHub/360capital/.venv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

# Récupération Crunchbase API pour récupérer le CSV initial

# Récupération des informations Affinity pour calculer le fundraising ratio

In [8]:
curl "https://api.affinity.co/api_endpoint" -u :$MXTzh9IZ0vry24Yd0qzSM0WHgxF7pzQHoKTzhyELlhw

SyntaxError: invalid syntax (2578553415.py, line 1)

In [9]:
import pandas as pd

# LSN pré filtre

In [11]:
df = pd.read_csv("/Users/justinkim/Documents/GitHub/360capital/extract_LSN - Feuille 1.csv")

In [12]:
df

,CompanyName,Website,Linkedin,FullName,Description,Industry,DurationInRole,Status,Notes,Search account,...,EmplyeeCount,Fullname,schoolname,schoolname2,schoolname3,CompanyName.1,CompanyName2,CompanyName3,Owner,Created
0,Ovadia,https://ovadia.rocks,https://www.linkedin.com/in/ACwAAABLbdIBnMNSRh...,Zohar Stolar,AI Automation for Ambitious Businesses\n \n At...,"Technology, Information and Internet",Experienced technical leader with a proven tra...,NaN,NaN,https://www.linkedin.com/company/ovadia/,...,0.0,Zohar Stolar,NaN,"RedHat Cert., Linux Networking and Security Ad...",NaN,France,France,France,NaN,14/10/2025
1,Parlenza.io,none.parlenza.io,https://www.linkedin.com/in/ACwAAAUp7vYBS9bwNo...,🌿 Boris Smets,NaN,"Technology, Information and Internet","Je suis lead développeur Full Stack , essentie...",NaN,NaN,https://www.linkedin.com/company/parlenza-io/,...,0.0,🌿 Boris Smets,NaN,"Master 2, modélisation numérique",NaN,NaN,"Nantes, Pays de la Loire, France",NaN,NaN,14/10/2025
2,AdhyaAI,adhyaai.com,https://www.linkedin.com/in/ACwAABVXP-gBYLDW0y...,Saumya Jetley,Single-person AI/ML consulting force founded t...,"Technology, Information and Internet",Led by curiosity and backed by rigor. * applie...,NaN,NaN,https://www.linkedin.com/company/adhyaai/,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14/10/2025
3,Koalya,none.koalya,https://www.linkedin.com/in/ACwAABKZe6oBdAic4x...,Baptiste Briot de La Crochais,Application collaborative qui connecte parents...,"Technology, Information and Internet",Looking for an internship,NaN,NaN,https://www.linkedin.com/company/koalya/,...,2.0,Baptiste Briot de La Crochais,https://media.licdn.com/dms/image/v2/C4D0BAQEW...,"Master degree, Business Law",https://www.linkedin.com/company/18223572/,"Aix-en-Provence, Provence-Alpes-Côte d’Azur, F...",Paris et périphérie,NaN,NaN,14/10/2025
4,Betterfolio,none.betterfolio,https://www.linkedin.com/in/ACwAAAtagtcBEX8RMM...,Junior Bernard,Betterfolio vous propose de génerer des dossie...,"Technology, Information and Internet","Moteur et dynamique, fort d’une experience ent...",NaN,NaN,https://www.linkedin.com/company/betterfolio/,...,2.0,Junior Bernard,https://media.licdn.com/dms/image/v2/C4D0BAQFS...,Bsc in Software development - License Génie Lo...,Teesside University,"Bordeaux, Nouvelle-Aquitaine, France","Bordeaux, Nouvelle-Aquitaine, France","Bordeaux, Nouvelle-Aquitaine, France",NaN,14/10/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,DGNUM - Direction générale du numérique,https://www.defense.gouv.fr/dgnum,https://www.linkedin.com/in/ACwAAAC17eQB_2CVcS...,Gauthier Monserand,Créée en 2018 à l’initiative de Mme Florence P...,Defense & Space,Passionate about teal organizations and softwa...,NaN,NaN,https://www.linkedin.com/company/dgnum/,...,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14/10/2025
329,Asayan,none.asayan,https://www.linkedin.com/in/ACwAAABjPhYBXXgBN0...,"Jean-Fabien RENARD, MBA",Notre solution rend vos processus immédiatemen...,Digital Accessibility Services,"Dans toutes les entreprises — industrie, servi...",NaN,NaN,https://www.linkedin.com/company/asayan-soluti...,...,2.0,"Jean-Fabien RENARD, MBA",https://media.licdn.com/dms/image/v2/D4D0BAQFW...,Université Paul Sabatier Toulouse III,NaN,"Toulouse, Occitanie, France",https://www.linkedin.com/sales/company/2217997,"Toulouse Area, France",NaN,14/10/2025
330,Swyp,www.swyp.be,https://www.linkedin.com/in/ACwAACkkabEBUKYdDD...,Dries Augustyns,"Because every smile is a celebration of life, ...",Dentists,I build solutions that solve problems.,NaN,NaN,https://www.linkedin.com/company/swypdentalshop/,...,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14/10/2025
331,HUMEAN,none.humean,https://www.linkedin.com/in/ACwAAAG68RgBGPJY1T...,Grégoire de Beaumont,"HUMEAN accompagne les dirigeants, DRH, DAF, re...",Insurance Agencies and Brokerages,"J’accompagne les dirigeants, DRH, DAF, représe...",NaN,NaN,https://www.linkedin.com/company/humean/,...,3.0,Grégoire de Beaumont,https://media.licdn

In [ ]:
import pandas as pd
from mistralai import Mistral

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [mistralai]13 [mistralai]


In [ ]:


def classify_company_status(df, client, model):
    """
    Classifie les entreprises et met 'X' dans la colonne Status si elles ne correspondent 
    pas aux critères (France/Italie, ou Europe + climate tech, pas de consulting).
    
    Args:
        df: DataFrame contenant les données
        client: Client Mistral initialisé
        model: Nom du modèle Mistral à utiliser
    
    Returns:
        DataFrame avec la colonne Status mise à jour
    """
    
    def should_exclude(row):
        """
        Détermine si une entreprise doit être exclue (Status = X)
        """
        if pd.isna(row.get('Description')) or str(row.get('Description')).strip() == '':
            return None
        
        description = str(row['Description'])
        prompt = f"""Analyse cette description d'entreprise et réponds UNIQUEMENT par 'EXCLURE' ou 'GARDER'.

Règles:
- GARDER si: l'entreprise est basée en France OU en Italie
- GARDER si: l'entreprise est en Europe ET travaille dans la climate tech (technologies climatiques, énergie renouvelable, décarbonation, etc.)
- EXCLURE si: l'entreprise fait du consulting/conseil
- EXCLURE si : l'entreprise est à but non lucratif ou une association

Description: {description}

Réponds uniquement par EXCLURE ou GARDER:"""
        
        try:
            chat_response = client.chat.complete(
                model=model,
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ]
            )
            
            response = chat_response.choices[0].message.content.strip().upper()
            

            if "EXCLURE" in response:
                return 'X'
            else:
                return None  
                
        except Exception as e:
            print(f"Erreur lors de la classification: {e}")
            return None
    

    print("Classification en cours...")
    for idx, row in df.iterrows():
        result = should_exclude(row)
        if result == 'X':
            df.at[idx, 'Status'] = 'X'
            if idx % 10 == 0: 
                print(f"Traité {idx + 1}/{len(df)} lignes")
    
    print("Classification terminée!")
    return df


# Config
api_key = "tLYewB74Gq1R7krnmU2fYaRVoHCx8wfl"
model = "mistral-small-latest"

client = Mistral(
    server_url="https://api.05d3a00300de.dc.mistral.ai",
    api_key=api_key
)

df_classified = classify_company_status(df, client, model)
df_classified.to_csv('companies_classified.csv', index=False)

Classification en cours...


/var/folders/k4/x11w534x6fx5843q4r1vdmtc0000gp/T/ipykernel_2753/3023613875.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'X' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'Status'] = 'X'


Traité 1/333 lignes
Traité 11/333 lignes
Traité 31/333 lignes
Traité 41/333 lignes
Traité 51/333 lignes
Traité 61/333 lignes
Traité 71/333 lignes
Traité 91/333 lignes
Traité 141/333 lignes
Traité 151/333 lignes
Traité 161/333 lignes
Traité 171/333 lignes
Traité 181/333 lignes
Traité 191/333 lignes
Traité 201/333 lignes
Traité 221/333 lignes
Traité 231/333 lignes
Traité 241/333 lignes
Traité 251/333 lignes
Traité 261/333 lignes
Traité 271/333 lignes
Traité 291/333 lignes
Traité 311/333 lignes
Traité 321/333 lignes
Traité 331/333 lignes
Classification terminée!


In [53]:
num = 2
while num < 10:
   print(num, end=" ")
   num **= 2

2 4 

In [44]:
2%2

0